In [1]:
import pandas as pd
import numpy as np

In [2]:
import os

#Define the path to the folder containing the CSV files
folder_path = 'idsdatasets'

#Get a list of all the CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

dfs = {}

#Loop through each CSV file and read it into a dataframe
for file in csv_files:
    df_name = os.path.splitext(file)[0]  # Get the dataframe name from the file name
    file_path = os.path.join(folder_path, file)  # Get the full file path
    df = pd.read_csv(file_path)  # Read the CSV file into a dataframe
    dfs[df_name] = df  # Assign the dataframe to a variable with the same name as the file (e.g. 'data1 = df1')

C:\Users\amenu4aie20110\AppData\Local\Temp\ipykernel_3152\826302849.py:15: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)  # Read the CSV file into a dataframe


KeyboardInterrupt: 

In [ ]:
dfs['02-20-2018'].drop(['Flow ID','Src IP','Dst IP'], axis=1, inplace=True)

In [ ]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key
    print(f"Dataframe: '{key}'\n")
    #count = df.values.ravel().tolist().count(np.inf)
    #count = np.sum(df.values == np.inf)
    #count2 = np.sum(df.values == -np.inf)
    #print(f"Inf:'{count}'\n -Inf:'{count2}'\n")
    # replace +ve and -ve infinity with NaN

    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    # Drop all NaN values
    df.dropna(inplace=True)

Dataframe: '02-14-2018'

Dataframe: '02-15-2018'

Dataframe: '02-16-2018'

Dataframe: '02-20-2018'

Dataframe: '02-21-2018'

Dataframe: '02-22-2018'

Dataframe: '02-23-2018'

Dataframe: '02-28-2018'

Dataframe: '03-01-2018'

Dataframe: '03-02-2018'



In [ ]:
labels = []
for i in dfs:
    for j in dfs[i]['Label'].unique():
        labels.append(j)
labels = list(set(labels))
labels

['DoS attacks-GoldenEye',
 'DoS attacks-SlowHTTPTest',
 'DDOS attack-LOIC-UDP',
 'Infilteration',
 'Benign',
 'SQL Injection',
 'SSH-Bruteforce',
 'Brute Force -Web',
 'Brute Force -XSS',
 'DDOS attack-HOIC',
 'Label',
 'DoS attacks-Slowloris',
 'Bot',
 'FTP-BruteForce',
 'DDoS attacks-LOIC-HTTP',
 'DoS attacks-Hulk']

In [ ]:
#labels.remove('Benign')
labels.remove("Label")
labels

['DoS attacks-GoldenEye',
 'DoS attacks-SlowHTTPTest',
 'DDOS attack-LOIC-UDP',
 'Infilteration',
 'SQL Injection',
 'SSH-Bruteforce',
 'Brute Force -Web',
 'Brute Force -XSS',
 'DDOS attack-HOIC',
 'DoS attacks-Slowloris',
 'Bot',
 'FTP-BruteForce',
 'DDoS attacks-LOIC-HTTP',
 'DoS attacks-Hulk']

In [ ]:
attackdfs = {}
for i in labels:
    df = pd.DataFrame()
    for j in dfs:
        if i in dfs[j]['Label'].unique():
            df = pd.concat([df, dfs[j][dfs[j]['Label'] == i]], axis=0)
            df.reset_index(inplace=True, drop=True)
    attackdfs[i] = df

In [9]:
attackdfs['DDoS attacks-LOIC-HTTP'].drop(['Src Port'], axis=1, inplace=True)

In [12]:
attackdfs['Benign'].drop(['Src Port'], axis=1, inplace=True)

KeyError: 'Benign'

In [10]:
attackdfs['Benign'] = attackdfs['Benign'].sample(frac=0.028).reset_index(drop=True)

KeyError: 'Benign'

In [11]:
for i in attackdfs:
    if "Src Port" in attackdfs[i].columns:
        print(i)

In [12]:
dfs = attackdfs

In [13]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key
    print(f"Dataframe: '{key}'\n")
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    # Drop all NaN values
    df.dropna(inplace=True)

Dataframe: 'DoS attacks-GoldenEye'

Dataframe: 'DoS attacks-SlowHTTPTest'

Dataframe: 'DDOS attack-LOIC-UDP'

Dataframe: 'Infilteration'

Dataframe: 'SQL Injection'

Dataframe: 'SSH-Bruteforce'

Dataframe: 'Brute Force -Web'

Dataframe: 'Brute Force -XSS'

Dataframe: 'DDOS attack-HOIC'

Dataframe: 'DoS attacks-Slowloris'

Dataframe: 'Bot'

Dataframe: 'FTP-BruteForce'

Dataframe: 'DDoS attacks-LOIC-HTTP'

Dataframe: 'DoS attacks-Hulk'



In [14]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key   
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%d/%m/%Y %H:%M:%S', errors='coerce')

In [15]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key   
    df['Timestamp'] = (df['Timestamp'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

In [16]:
#Iterate throught each dataframe in the dictionary
for key in dfs.keys():
    df = dfs[key]
    for col in df.columns:
        #Check if the datatype of the column is object
        if df[col].dtype == 'object' and col != 'Label':
            # Change all values to numeric, and to NaN if it is a strig
            df[col] = pd.to_numeric(df[col], errors='coerce')

In [17]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key
    print(f"Dataframe: '{key}'\n")
    count_NA = df.isna().sum()
    print(count_NA)

Dataframe: 'DoS attacks-GoldenEye'

Dst Port         0
Protocol         0
Timestamp        0
Flow Duration    0
Tot Fwd Pkts     0
                ..
Idle Mean        0
Idle Std         0
Idle Max         0
Idle Min         0
Label            0
Length: 80, dtype: int64
Dataframe: 'DoS attacks-SlowHTTPTest'

Dst Port         0
Protocol         0
Timestamp        0
Flow Duration    0
Tot Fwd Pkts     0
                ..
Idle Mean        0
Idle Std         0
Idle Max         0
Idle Min         0
Label            0
Length: 80, dtype: int64
Dataframe: 'DDOS attack-LOIC-UDP'

Dst Port         0
Protocol         0
Timestamp        0
Flow Duration    0
Tot Fwd Pkts     0
                ..
Idle Mean        0
Idle Std         0
Idle Max         0
Idle Min         0
Label            0
Length: 80, dtype: int64
Dataframe: 'Infilteration'

Dst Port         0
Protocol         0
Timestamp        0
Flow Duration    0
Tot Fwd Pkts     0
                ..
Idle Mean        0
Idle Std         0
Idle Max

In [18]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key
    df.dropna(inplace=True)

In [19]:
from scipy import stats

for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key

    # Define a function to filter outliers using Z-score
    def filter_outliers_zscore(data, threshold):
        z_scores = np.abs(stats.zscore(data))
        outlier_mask = (z_scores > threshold).any(axis=1)
        return data[~outlier_mask], data[outlier_mask]

    # Define a threshold value
    threshold = 7


    # The filtering was removing all DDoS-LOIC-UDP, so we will not execute it on them

    if key == 'DDOS attack-LOIC-UDP':
        df_temp = df[df['Label'] == 'DDOS attack-LOIC-UDP']
        df = df[df['Label'] != 'DDOS attack-LOIC-UDP']
        



    # Loop through the columns of the dataframe and filter outliers in each column
    filtered_cols = []
    removed_outliers = []
    for col in df.columns:
        if col != 'Label':
            filtered_col, outliers = filter_outliers_zscore(df[[col]], threshold)
            
            filtered_cols.append(filtered_col)
            removed_outliers.append(outliers)

    # Combine the filtered columns back into a dataframe
    df_filtered = pd.concat(filtered_cols, axis=1)

    # Combine the removed outliers back into a dataframe
    df_outliers = pd.concat(removed_outliers, axis=1)

    # Dataframe filtering comparison

    print(f'\nDataframe: {key} Shape: {df.shape}')

    # Print the number of outliers removed for each column
    
    print('Outlier removal summary:')
    n_outliers = df_outliers.shape[0]
    print(f'{n_outliers} outliers rows to be removed')

    # Print the original dataframe and the filtered dataframe side by side
    print('\nOriginal dataframe:')
    display(df.head())
    
    # Assign filtered dataframe columns to original one
    columns = [col for col in df.columns if col != 'Label']
    df.loc[:,columns] = df_filtered.loc[:,columns]
    

    # Recombine rows from 'DDOS attack-LOIC-UDP'
    if key == 'DDOS attack-LOIC-UDP':
        df = pd.concat([df,df_temp])


    print('\nFiltered dataframe:')
    display(df.head())

    # Print the removed outliers dataframe
    print('\nRemoved outliers:')
    display(df_outliers.head())

    # Count how many rows of each attack were removed from original dataframe
    values_orig = df.loc[df.index.isin(df_outliers.index), 'Label']
    print(f'\n{values_orig.value_counts()}')


Dataframe: DoS attacks-GoldenEye Shape: (41508, 80)
Outlier removal summary:
840 outliers rows to be removed

Original dataframe:


,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,80,6,1518686866,6010454,4,4,285,972,285,0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,DoS attacks-GoldenEye
1,80,6,1518686866,6005042,4,4,422,662,422,0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,DoS attacks-GoldenEye
2,80,6,1518686866,6003639,4,4,548,972,548,0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,DoS attacks-GoldenEye
3,80,6,1518686866,5998377,4,4,327,972,327,0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,DoS attacks-GoldenEye
4,80,6,1518686866,5999806,4,4,378,972,378,0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,DoS attacks-GoldenEye



Filtered dataframe:


,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,80,6,1518686866,6010454,4.0,4,285.0,972,285,0.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,DoS attacks-GoldenEye
1,80,6,1518686866,6005042,4.0,4,422.0,662,422,0.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,DoS attacks-GoldenEye
2,80,6,1518686866,6003639,4.0,4,548.0,972,548,0.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,DoS attacks-GoldenEye
3,80,6,1518686866,5998377,4.0,4,327.0,972,327,0.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,DoS attacks-GoldenEye
4,80,6,1518686866,5999806,4.0,4,378.0,972,378,0.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,DoS attacks-GoldenEye



Removed outliers:


...


Filtered dataframe:


,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,80,6,1518745527,1793.0,3.0,4,364.0,935,364,0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS attacks-Hulk
1,80,6,1518745527,1720.0,3.0,4,300.0,935,300,0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS attacks-Hulk
2,80,6,1518745527,191.0,2.0,0,0.0,0,0,0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS attacks-Hulk
3,80,6,1518745527,912.0,3.0,4,359.0,935,359,0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS attacks-Hulk
4,80,6,1518745527,1869.0,3.0,4,384.0,935,384,0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS attacks-Hulk



Removed outliers:


,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
156150,NaN,NaN,NaN,61253487.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,61200000.0,NaN,61200000.0,61200000.0
156199,NaN,NaN,NaN,61266477.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,61200000.0,NaN,61200000.0,61200000.0
156398,NaN,NaN,NaN,61249974.0,6.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,61100000.0,NaN,61100000.0,61100000.0
158521,NaN,NaN,NaN,62236061.0,7.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,61200000.0,NaN,61200000.0,61200000.0
158628,NaN,NaN,NaN,61013945.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,61000000.0,NaN,61000000.0,61000000.0



Label
DoS attacks-Hulk    6262
Name: count, dtype: int64


In [22]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key
    print(f"Dataframe: '{key}'\n")
    count_NA = df.isna().sum()
    print(count_NA)

Dataframe: 'FTP-BruteForce'

Dst Port          0
Protocol          0
Timestamp         0
Flow Duration    23
Tot Fwd Pkts      0
                 ..
Idle Mean         0
Idle Std          0
Idle Max          0
Idle Min          0
Label             0
Length: 80, dtype: int64
Dataframe: 'DoS attacks-Slowloris'

Dst Port         0
Protocol         0
Timestamp        0
Flow Duration    0
Tot Fwd Pkts     0
                ..
Idle Mean        0
Idle Std         0
Idle Max         0
Idle Min         0
Label            0
Length: 80, dtype: int64
Dataframe: 'DoS attacks-GoldenEye'

Dst Port           0
Protocol           0
Timestamp          0
Flow Duration      0
Tot Fwd Pkts       4
                ... 
Idle Mean          0
Idle Std         174
Idle Max           0
Idle Min           0
Label              0
Length: 80, dtype: int64
Dataframe: 'DDOS attack-LOIC-UDP'

Dst Port         0
Protocol         0
Timestamp        0
Flow Duration    0
Tot Fwd Pkts     0
                ..
Idle Mean      

In [20]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key
    df.dropna(inplace=True)

In [21]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key
    print(df.shape)

(40668, 80)
(139877, 80)
(1730, 80)
(152980, 80)
(86, 80)
(187469, 80)
(600, 80)
(219, 80)
(683607, 80)
(10468, 80)
(280820, 80)
(193331, 80)
(573061, 80)
(455650, 80)


In [22]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key
    print(f"Dataframe: '{key}'\n")
    # replace +ve and -ve infinity with NaN
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    # Drop all NaN values
    df.dropna(inplace=True)

Dataframe: 'DoS attacks-GoldenEye'

Dataframe: 'DoS attacks-SlowHTTPTest'

Dataframe: 'DDOS attack-LOIC-UDP'

Dataframe: 'Infilteration'

Dataframe: 'SQL Injection'

Dataframe: 'SSH-Bruteforce'

Dataframe: 'Brute Force -Web'

Dataframe: 'Brute Force -XSS'

Dataframe: 'DDOS attack-HOIC'

Dataframe: 'DoS attacks-Slowloris'

Dataframe: 'Bot'

Dataframe: 'FTP-BruteForce'

Dataframe: 'DDoS attacks-LOIC-HTTP'

Dataframe: 'DoS attacks-Hulk'



In [26]:
from sklearn.preprocessing import MinMaxScaler
for key in dfs.keys():
       df = dfs[key]  # Get the dataframe corresponding to the key
       columns = [col for col in df.columns if col != 'Label']
       min_max_scaler = MinMaxScaler().fit(df[columns])
       print(f"Dataframe: {key}\n")
       df[columns] = min_max_scaler.transform(df[columns])
       display(df.head())

Dataframe: FTP-BruteForce



,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.0,0.0,-1.73197,2.615952,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,FTP-BruteForce
1,0.0,0.0,-1.73197,-0.137321,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,FTP-BruteForce
2,0.0,0.0,-1.73197,-0.137321,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,FTP-BruteForce
3,0.0,0.0,-1.73197,-0.309401,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,FTP-BruteForce
4,0.0,0.0,-1.73197,-0.309401,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,FTP-BruteForce


Dataframe: DoS attacks-Slowloris



,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.0,0.0,-1.630817,-1.487619,0.042065,0.043305,-0.147790,0.0,1.080457,-0.662943,...,-0.520582,-0.876766,-0.84771,-0.873992,-0.877634,-0.903389,-0.87316,-1.26034,-0.666383,DoS attacks-Slowloris
1,0.0,0.0,-1.626676,-1.547415,-0.604537,0.043305,-0.704423,0.0,1.080457,-0.662943,...,-0.520582,-0.876766,-0.84771,-0.873992,-0.877634,-0.903389,-0.87316,-1.26034,-0.666383,DoS attacks-Slowloris
2,0.0,0.0,-1.625295,-1.551481,-0.281236,0.043305,-0.147790,0.0,1.080457,-0.662943,...,-0.520582,-0.876766,-0.84771,-0.873992,-0.877634,-0.903389,-0.87316,-1.26034,-0.666383,DoS attacks-Slowloris
3,0.0,0.0,-1.618393,-1.568958,-0.766187,0.043305,-0.704423,0.0,1.080457,-0.662943,...,-0.520582,-0.876766,-0.84771,-0.873992,-0.877634,-0.903389,-0.87316,-1.26034,-0.666383,DoS attacks-Slowloris
4,0.0,0.0,-1.618393,-1.568947,-0.766187,0.043305,-0.704423,0.0,1.080457,-0.662943,...,-0.520582,-0.876766,-0.84771,-0.873992,-0.877634,-0.903389,-0.87316,-1.26034,-0.666383,DoS attacks-Slowloris


Dataframe: DoS attacks-GoldenEye



...

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
2,0.0,0.0,-1.426941,0.894878,0.982771,1.113920,0.959559,1.074030,1.013794,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Brute Force -XSS
3,0.0,0.0,-1.425588,-0.982569,-0.988207,-0.983113,-0.983084,-0.983090,-0.986394,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Brute Force -XSS
5,0.0,0.0,-1.424685,-0.982573,-0.988207,-0.983113,-0.983084,-0.983090,-0.986394,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Brute Force -XSS
6,0.0,0.0,-1.424811,0.951101,0.982771,1.113920,0.962951,1.076296,1.013794,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Brute Force -XSS
7,0.0,0.0,-1.423432,-0.982570,-0.988207,-0.983113,-0.983084,-0.983090,-0.986394,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Brute Force -XSS


Dataframe: SSH-Bruteforce



,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.0,0.0,-1.73052,2.016120,0.956773,1.066397,0.993373,0.99997,0.999941,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SSH-Bruteforce
1,0.0,0.0,-1.73052,2.042842,0.956773,1.066397,1.009915,0.99997,0.999941,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SSH-Bruteforce
2,0.0,0.0,-1.73052,1.880194,0.956773,0.869791,0.993373,0.99997,0.999941,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SSH-Bruteforce
3,0.0,0.0,-1.73052,1.896774,0.956773,0.968094,1.026457,0.99997,0.999941,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SSH-Bruteforce
4,0.0,0.0,-1.73052,2.069288,0.956773,0.968094,1.009915,0.99997,0.999941,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SSH-Bruteforce


Dataframe: Infilteration



,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,-0.429981,-0.692652,-0.839336,-0.331197,-0.158837,-0.466566,-0.263774,-0.209702,-0.375192,-0.68289,...,0.4232,-0.17375,-0.117331,-0.189746,-0.155122,-0.274155,-0.146438,-0.278402,-0.265963,Infilteration
1,-0.429981,-0.692652,-0.839288,-0.331196,-0.158837,-0.466566,-0.263774,-0.209702,-0.375192,-0.68289,...,0.4232,-0.17375,-0.117331,-0.189746,-0.155122,-0.274155,-0.146438,-0.278402,-0.265963,Infilteration
2,-0.429981,-0.692652,-0.839288,-0.331183,-0.158837,-0.466566,-0.263774,-0.209702,-0.375192,-0.68289,...,0.4232,-0.17375,-0.117331,-0.189746,-0.155122,-0.274155,-0.146438,-0.278402,-0.265963,Infilteration
3,-0.429981,-0.692652,-0.839288,-0.331177,-0.158837,-0.466566,-0.263774,-0.209702,-0.375192,-0.68289,...,0.4232,-0.17375,-0.117331,-0.189746,-0.155122,-0.274155,-0.146438,-0.278402,-0.265963,Infilteration
4,-0.429981,-0.692652,-0.839288,-0.331174,-0.158837,-0.466566,-0.263774,-0.209702,-0.375192,-0.68289,...,0.4232,-0.17375,-0.117331,-0.189746,-0.155122,-0.274155,-0.146438,-0.278402,-0.265963,Infilteration


In [27]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key


    columns = [col for col in df.columns if col != 'Label']

    corr_matrix = df[columns].corr().abs()

    threshold = 0.99
    # Find features with high correlation
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

    print(f'\nDataset: {key}')
    # Print features to drop
    print(f"The following {len(to_drop)} features will be dropped due to high correlation: {to_drop}")

    df = df.drop(to_drop, axis = 1)

'for key in dfs.keys():\n    df = dfs[key]  # Get the dataframe corresponding to the key\n\n\n    columns = [col for col in df.columns if col != \'Label\']\n\n    corr_matrix = df[columns].corr().abs()\n\n    threshold = 0.99\n    # Find features with high correlation\n    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))\n    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]\n\n    print(f\'\nDataset: {key}\')\n    # Print features to drop\n    print(f"The following {len(to_drop)} features will be dropped due to high correlation: {to_drop}")\n\n    df = df.drop(to_drop, axis = 1)'

In [23]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key
    print(f"Dataframe: '{key}'\n")
    # replace +ve and -ve infinity with NaN
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    # Drop all NaN values
    df.dropna(inplace=True)

Dataframe: 'DoS attacks-GoldenEye'

Dataframe: 'DoS attacks-SlowHTTPTest'

Dataframe: 'DDOS attack-LOIC-UDP'

Dataframe: 'Infilteration'

Dataframe: 'SQL Injection'

Dataframe: 'SSH-Bruteforce'

Dataframe: 'Brute Force -Web'

Dataframe: 'Brute Force -XSS'

Dataframe: 'DDOS attack-HOIC'

Dataframe: 'DoS attacks-Slowloris'

Dataframe: 'Bot'

Dataframe: 'FTP-BruteForce'

Dataframe: 'DDoS attacks-LOIC-HTTP'

Dataframe: 'DoS attacks-Hulk'



In [31]:
"""for i in dfs:
    dfs[i].to_csv('temp/' + i + '.csv')"""

"for i in dfs:\n    dfs[i].to_csv('temp/' + i + '.csv')"

In [32]:
for i in dfs:
    print(dfs[i].shape)

(193331, 80)
(10468, 80)
(40668, 80)
(1730, 80)
(139877, 80)
(573061, 80)
(280820, 80)
(455650, 80)
(600, 80)
(683607, 80)
(86, 80)
(219, 80)
(187469, 80)
(152880, 80)


In [33]:
for key in dfs.keys():
    df = dfs[key]  # Get the dataframe corresponding to the key
    print(f"Dataframe: '{key}'\n")
    print(f"{df['Label'].value_counts()}\n")

Dataframe: 'FTP-BruteForce'

Label
FTP-BruteForce    193331
Name: count, dtype: int64

Dataframe: 'DoS attacks-Slowloris'

Label
DoS attacks-Slowloris    10468
Name: count, dtype: int64

Dataframe: 'DoS attacks-GoldenEye'

Label
DoS attacks-GoldenEye    40668
Name: count, dtype: int64

Dataframe: 'DDOS attack-LOIC-UDP'

Label
DDOS attack-LOIC-UDP    1730
Name: count, dtype: int64

Dataframe: 'DoS attacks-SlowHTTPTest'

Label
DoS attacks-SlowHTTPTest    139877
Name: count, dtype: int64

Dataframe: 'DDoS attacks-LOIC-HTTP'

Label
DDoS attacks-LOIC-HTTP    573061
Name: count, dtype: int64

Dataframe: 'Bot'

Label
Bot    280820
Name: count, dtype: int64

Dataframe: 'DoS attacks-Hulk'

Label
DoS attacks-Hulk    455650
Name: count, dtype: int64

Dataframe: 'Brute Force -Web'

Label
Brute Force -Web    600
Name: count, dtype: int64

Dataframe: 'DDOS attack-HOIC'

Label
DDOS attack-HOIC    683607
Name: count, dtype: int64

Dataframe: 'SQL Injection'

Label
SQL Injection    86
Name: count, dtyp

In [24]:
combined = pd.concat(dfs.values())

In [26]:
for column in combined.columns:
    if (combined[column] == 0).all():
        combined[column] = combined[column].replace(0,np.nan)

combined=combined.dropna(axis=1,how='all')

In [27]:
combined.describe()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,2.720466e+06,2.720466e+06,2.720466e+06,2.720466e+06,2.720466e+06,2.720466e+06,2.720466e+06,2.720466e+06,2.720466e+06,2.720466e+06,...,2.720466e+06,2.720466e+06,2.720466e+06,2.720466e+06,2.720466e+06,2.720466e+06,2.720466e+06,2.720466e+06,2.720466e+06,2.720466e+06
mean,2.023726e-02,3.653551e-01,3.721147e-01,2.994213e-02,2.473727e-04,7.639355e-03,2.546201e-04,1.081741e-03,5.033937e-02,1.635987e-03,...,2.439383e-04,6.335467e-01,7.843220e-04,4.054994e-04,7.687191e-04,7.382951e-04,2.294270e-02,1.943193e-03,2.340152e-02,2.264067e-02
std,7.574945e-02,8.946170e-02,2.969051e-01,1.066921e-01,9.827859e-03,1.692552e-02,9.827946e-03,5.894048e-03,1.124610e-01,1.332500e-02,...,9.827938e-03,1.949965e-01,1.360084e-02,8.317825e-03,1.287854e-02,1.320790e-02,8.081879e-02,4.058609e-02,8.242559e-02,8.043329e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.220796e-03,3.529412e-01,1.209103e-01,5.375000e-06,3.229682e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.220796e-03,3.529412e-01,3.868839e-01,8.906667e-05,3.229682e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.220796e-03,3.529412e-01,4.262358e-01,3.271581e-03,6.459364e-06,1.498127e-02,2.018545e-06,4.089020e-04,1.369863e-02,0.000000e+00,...,3.229682e-06,8.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [28]:
combined['Label'].value_counts()

Label
DDOS attack-HOIC            683607
DDoS attacks-LOIC-HTTP      573061
DoS attacks-Hulk            455650
Bot                         280820
FTP-BruteForce              193331
SSH-Bruteforce              187469
Infilteration               152880
DoS attacks-SlowHTTPTest    139877
DoS attacks-GoldenEye        40668
DoS attacks-Slowloris        10468
DDOS attack-LOIC-UDP          1730
Brute Force -Web               600
Brute Force -XSS               219
SQL Injection                   86
Name: count, dtype: int64

In [29]:
combined.shape

(2720466, 69)

In [30]:
# Assuming 'combined' is your DataFrame and you want to save it to a CSV file
combined.to_csv('temp/combined.csv')